In [2]:
!pip install faker

  Using cached faker-37.4.2-py3-none-any.whl.metadata (15 kB)
Using cached faker-37.4.2-py3-none-any.whl (1.9 MB)


In [3]:
from faker import Faker

fake = Faker('es_ES')
print(fake.name())

Victoriano Torrecilla Posada


In [21]:
pip install pymysql

Note: you may need to restart the kernel to use updated packages.


In [23]:
# ----------------- IMPORTAR LIBRERÍAS -----------------
import pandas as pd
import random
from faker import Faker
from datetime import datetime
from sqlalchemy import create_engine

# ----------------- CONFIGURACIÓN -----------------
fake = Faker('es_ES')  # Generador en español

# Rutas a tus archivos CSV locales
categorias_df = pd.read_csv(r"C:\Users\Usuario\OneDrive\Vs code\superMercado\categorias.csv")
proveedores_df = pd.read_csv(r"C:\Users\Usuario\OneDrive\Vs code\superMercado\proveedores.csv")

# Definir grupos de productos y cantidades
grupo_productos = {
    "Abarrotes": (1600, 2400),
    "Lácteos y Huevos": (960, 1440),
    "Carnes y Aves": (800, 1200),
    "Frutas y Verduras": (960, 1440),
    "Bebidas": (1200, 1800),
    "Limpieza del Hogar": (800, 1200),
    "Congelados": (640, 960),
    "Panadería y Repostería": (400, 600),
    "Cuidado Personal": (400, 600),
    "Otros": (240, 360)
}

# Relación entre grupos y categorías
grupo_categoria_map = {
    "Abarrotes": ["Aceites y Vinagres", "Arroz y Legumbres", "Pastas y Salsas", "Enlatados", "Condimentos", "Cereales y Granos"],
    "Lácteos y Huevos": ["Lácteos", "Huevos", "Quesos"],
    "Carnes y Aves": ["Carnes Rojas", "Aves", "Pescados Frescos", "Mariscos"],
    "Frutas y Verduras": ["Frutas Frescas", "Verduras y Hortalizas"],
    "Bebidas": ["Bebidas No Alcohólicas", "Vinos y Licores", "Cervezas", "Café y Té"],
    "Limpieza del Hogar": ["Artículos de Limpieza"],
    "Congelados": ["Alimentos Congelados"],
    "Panadería y Repostería": ["Panadería Fresca", "Repostería", "Tortillas y Masas"],
    "Cuidado Personal": ["Cuidado Personal"],
    "Otros": ["Mascotas", "Productos para Bebés", "Alimentos Internacionales", "Snacks Saludables", "Chocolates y Dulces"]
}

# ----------------- GENERAR PRODUCTOS -----------------
productos = []

for grupo, (min_p, max_p) in grupo_productos.items():
    cantidad = random.randint(min_p, max_p)
    categorias_posibles = grupo_categoria_map[grupo]
    
    for _ in range(cantidad):
        nombre = f"{fake.word().capitalize()} {fake.color_name()} {random.choice(['500g', '1L', 'pack', 'bolsa'])}"
        descripcion = fake.sentence(nb_words=10)
        precio_unitario = round(random.uniform(1.0, 100.0), 2)
        stock = random.randint(0, 100)
        stock_minimo = random.randint(5, 20)

        categoria_nombre = random.choice(categorias_posibles)
        categoria_row = categorias_df[categorias_df['nombre'].str.contains(categoria_nombre, case=False, na=False)]
        id_categoria = int(categoria_row.sample(1).iloc[0]['id_categoria']) if not categoria_row.empty else int(categorias_df.sample(1).iloc[0]['id_categoria'])

        id_proveedor = int(proveedores_df.sample(1).iloc[0]['id_proveedor'])
        fecha_creacion = fake.date_between(start_date='-2y', end_date='today')

        productos.append({
            "nombre": nombre,
            "descripcion": descripcion,
            "precio_unitario": precio_unitario,
            "stock": stock,
            "stock_minimo": stock_minimo,
            "id_categoria": id_categoria,
            "id_proveedor": id_proveedor,
            "fecha_creacion": fecha_creacion
        })

# Crear DataFrame
df = pd.DataFrame(productos)

# ----------------- CONEXIÓN MYSQL CON PYMYSQL -----------------
from sqlalchemy import text  # ✅ IMPORTAR text

usuario = "root"
contraseña = "Datasciencie98"
host = "localhost"
base_datos = "supermercado"

# Conexión usando pymysql
conexion = create_engine(f"mysql+pymysql://{usuario}:{contraseña}@{host}/{base_datos}")

# ----------------- ELIMINAR DATOS ANTERIORES -----------------
with conexion.begin() as conn:
    conn.execute(text("DELETE FROM productos"))  # ✅ ENVUELTO EN text()

# ----------------- INSERTAR NUEVOS DATOS -----------------
df.to_sql('productos', con=conexion, if_exists='append', index=False)

print(f"✅ Productos generados: {len(df)}")
print("🚀 Tabla 'productos' actualizada correctamente en español.")


✅ Productos generados: 9308
🚀 Tabla 'productos' actualizada correctamente en español.


In [24]:
import pandas as pd
from faker import Faker
import random
from sqlalchemy import create_engine, text
from tqdm import tqdm
import time

# Inicializar Faker en español
faker = Faker('es_ES')
Faker.seed(42)

# Configuración general
total_clientes = 1_000_000
batch_size = 50_000
ruta_csv = r"C:\Users\Usuario\OneDrive\Vs code\superMercado\clientes.csv"

# Lista para acumular todos los clientes
clientes_totales = []

# Función para generar lote
def generar_lote(n):
    clientes = []
    for _ in range(n):
        nombre = faker.name()
        direccion = faker.address().replace('\n', ', ')
        telefono = faker.phone_number()
        email = faker.unique.email()
        puntos = random.randint(0, 5000)
        clientes.append({
            "nombre": nombre,
            "direccion": direccion,
            "telefono": telefono,
            "email": email,
            "puntos_acumulados": puntos
        })
    return clientes

# Generar todos los datos por lotes y guardar en lista
print("🚀 Generando datos...")
for i in tqdm(range(0, total_clientes, batch_size)):
    clientes_totales.extend(generar_lote(batch_size))

# Crear DataFrame
df = pd.DataFrame(clientes_totales)

# Guardar en CSV
df.to_csv(ruta_csv, index=False)
print(f"📁 Archivo CSV generado: {ruta_csv}")

# Conexión a MySQL
usuario = "root"
contraseña = "Datasciencie98"
host = "localhost"
base_datos = "supermercado"
engine = create_engine(f"mysql+pymysql://{usuario}:{contraseña}@{host}/{base_datos}")

# Borrar datos anteriores
print("🧹 Eliminando datos anteriores...")
with engine.begin() as conn:
    conn.execute(text("DELETE FROM clientes"))

# Insertar en MySQL por lotes
print("📤 Subiendo a MySQL...")
inicio = time.time()
for i in range(0, len(df), batch_size):
    df_lote = df.iloc[i:i+batch_size]
    df_lote.to_sql('clientes', con=engine, if_exists='append', index=False)
    print(f"✅ Insertado lote: {i+1} a {i+len(df_lote)}")

fin = time.time()
print(f"\n🎉 Se generaron e insertaron {len(df)} clientes correctamente en {(fin - inicio)/60:.2f} minutos.")


🚀 Generando datos...


100%|██████████| 20/20 [06:30<00:00, 19.52s/it]


📁 Archivo CSV generado: C:\Users\Usuario\OneDrive\Vs code\superMercado\clientes.csv
🧹 Eliminando datos anteriores...
📤 Subiendo a MySQL...
✅ Insertado lote: 1 a 50000
✅ Insertado lote: 50001 a 100000
✅ Insertado lote: 100001 a 150000
✅ Insertado lote: 150001 a 200000
✅ Insertado lote: 200001 a 250000
✅ Insertado lote: 250001 a 300000
✅ Insertado lote: 300001 a 350000
✅ Insertado lote: 350001 a 400000
✅ Insertado lote: 400001 a 450000
✅ Insertado lote: 450001 a 500000
✅ Insertado lote: 500001 a 550000
✅ Insertado lote: 550001 a 600000
✅ Insertado lote: 600001 a 650000
✅ Insertado lote: 650001 a 700000
✅ Insertado lote: 700001 a 750000
✅ Insertado lote: 750001 a 800000
✅ Insertado lote: 800001 a 850000
✅ Insertado lote: 850001 a 900000
✅ Insertado lote: 900001 a 950000
✅ Insertado lote: 950001 a 1000000

🎉 Se generaron e insertaron 1000000 clientes correctamente en 1.69 minutos.


In [26]:
import pandas as pd
import random
from faker import Faker
from datetime import timedelta
from sqlalchemy import create_engine

# Inicializar Faker en español
fake = Faker('es_ES')

# Rutas locales de tus archivos descargados
productos_df = pd.read_csv(r"C:\Users\Usuario\OneDrive\Vs code\superMercado\productos.csv")
categorias_df = pd.read_csv(r"C:\Users\Usuario\OneDrive\Vs code\superMercado\categorias.csv")

# Lista para almacenar promociones
promociones = []

# Generar 20,000 promociones
for _ in range(20000):
    nombre = f"Promo {fake.word().capitalize()} {random.choice(['10%', '20%', '2x1', 'Oferta'])}"
    descripcion = fake.sentence(nb_words=10)
    descuento = round(random.uniform(5, 50), 2)

    fecha_inicio = fake.date_between(start_date='-1y', end_date='today')
    fecha_fin = fecha_inicio + timedelta(days=random.randint(5, 60))

    producto = productos_df.sample(1).iloc[0]
    id_producto = int(producto["id_producto"])
    id_categoria = int(producto["id_categoria"])

    promociones.append({
        "nombre": nombre,
        "descripcion": descripcion,
        "descuento": descuento,
        "fecha_inicio": fecha_inicio,
        "fecha_fin": fecha_fin,
        "id_producto": id_producto,
        "id_categoria": id_categoria
    })

# Crear DataFrame
promociones_df = pd.DataFrame(promociones)

# Guardar en CSV (opcional)
promociones_df.to_csv(r"C:\Users\Usuario\OneDrive\Vs code\superMercado\promociones.csv", index=False)

# Subir directamente a MySQL
usuario = "root"
contraseña = "Datasciencie98"
host = "localhost"
base_datos = "supermercado"

# Cambia "mysqlconnector" por "pymysql" si tu driver es pymysql
conexion = create_engine(f"mysql+pymysql://{usuario}:{contraseña}@{host}/{base_datos}")

# Opcional: Eliminar datos anteriores
with conexion.begin() as conn:
    conn.execute(text("DELETE FROM promociones"))

# Insertar nuevos
promociones_df.to_sql('promociones', con=conexion, if_exists='append', index=False)

print(f"✅ Promociones generadas: {len(promociones_df)}")
print("🚀 Tabla 'promociones' actualizada en MySQL.")


✅ Promociones generadas: 20000
🚀 Tabla 'promociones' actualizada en MySQL.


In [31]:
pip install sqlalchemy pymysql

In [ ]:
import pandas as pd
import numpy as np
from faker import Faker
import random
from itertools import product
from sqlalchemy import create_engine, text

# -------------------- CONFIGURACIÓN --------------------
fake = Faker('es_ES')

# Configuración de archivos y MySQL (¡ajusta rutas!)
config = {
    "csv_productos": r"C:\Users\Usuario\OneDrive\Vs code\superMercado\productos.csv",
    "csv_sucursales": r"C:\Users\Usuario\OneDrive\Vs code\superMercado\sucursales.csv",
    "csv_inventario": r"C:\Users\Usuario\OneDrive\Vs code\superMercado\inventario_generado.csv",
    "mysql_user": "root",
    "mysql_password": "Datasciencie98",
    "mysql_host": "localhost",
    "mysql_db": "supermercado"
}

# -------------------- FUNCIONES --------------------
def cargar_datos(ruta_csv, dtype, id_col):
    """Carga CSV y valida que tenga datos."""
    df = pd.read_csv(ruta_csv, dtype=dtype)
    print(f"📊 {len(df)} registros cargados desde {ruta_csv}")
    assert not df.empty, f"❌ El archivo {ruta_csv} está vacío"
    assert id_col in df.columns, f"❌ Falta columna {id_col} en {ruta_csv}"
    return df

def generar_inventario(productos_df, sucursales_df):
    """Genera el DataFrame de inventario con todas las combinaciones."""
    productos = productos_df['id_producto'].unique()
    sucursales = sucursales_df['id_sucursal'].unique()
    
    # Generar todas las combinaciones producto-sucursal
    combinaciones = list(product(productos, sucursales))
    random.shuffle(combinaciones)
    
    # Validar volumen esperado (9,308 productos × 15 sucursales = 139,620)
    assert len(combinaciones) == 139_620, \
        f"❌ Combinaciones incorrectas: {len(combinaciones)} (esperado: 139,620)"
    
    # Asignar cantidades basadas en precio
    precio_dict = productos_df.set_index('id_producto')['precio_unitario'].to_dict()
    def cantidad_por_precio(precio):
        return (
            random.randint(50, 500) if precio < 20 else
            random.randint(20, 200) if precio < 50 else
            random.randint(0, 50)
        )
    
    # Crear DataFrame
    inventario = pd.DataFrame({
        'id_producto': [c[0] for c in combinaciones],
        'id_sucursal': [c[1] for c in combinaciones],
        'cantidad': [cantidad_por_precio(precio_dict[c[0]]) for c in combinaciones],
        'fecha_actualizacion': [fake.date_between(start_date='-90d', end_date='today') 
                              for _ in combinaciones]
    })
    
    # Verificar duplicados
    assert not inventario.duplicated(subset=['id_producto', 'id_sucursal']).any(), \
        "❌ Hay registros duplicados en producto-sucursal"
    
    return inventario

# -------------------- EJECUCIÓN PRINCIPAL --------------------
try:
    # 1. Cargar datos base
    productos_df = cargar_datos(
        config["csv_productos"],
        dtype={'id_producto': 'int32', 'precio_unitario': 'float32'},
        id_col='id_producto'
    )
    sucursales_df = cargar_datos(
        config["csv_sucursales"],
        dtype={'id_sucursal': 'int32'},
        id_col='id_sucursal'
    )
    
    # 2. Generar inventario
    print("⚙️ Generando inventario...")
    inventario_df = generar_inventario(productos_df, sucursales_df)
    
    # 3. Exportar a CSV
    inventario_df.to_csv(config["csv_inventario"], index=False, encoding='utf-8-sig')
    print(f"💾 CSV guardado en: {config['csv_inventario']}")
    
    # 4. Cargar a MySQL
    engine = create_engine(
        f"mysql+pymysql://{config['mysql_user']}:{config['mysql_password']}@"
        f"{config['mysql_host']}/{config['mysql_db']}",
        pool_pre_ping=True
    )
    
    with engine.begin() as conn:
        print("🧹 Limpiando tabla existente...")
        conn.execute(text("TRUNCATE TABLE inventario"))
        
        print("⬆️ Subiendo datos a MySQL...")
        inventario_df.to_sql(
            'inventario',
            con=conn,
            if_exists='append',
            index=False,
            chunksize=10_000,
            method='multi'
        )
    
    print(f"✅ ¡Proceso completado! Registros insertados: {len(inventario_df)}")

except Exception as e:
    print(f"❌ Error: {str(e)}")
    # Opcional: guardar log de errores
    with open("error_log.txt", "a") as f:
        f.write(f"{pd.Timestamp.now()}: {str(e)}\n")
        

📊 1000 registros cargados desde C:\Users\Usuario\OneDrive\Vs code\superMercado\productos.csv
📊 15 registros cargados desde C:\Users\Usuario\OneDrive\Vs code\superMercado\sucursales.csv
⚙️ Generando inventario...
❌ Error: ❌ Combinaciones incorrectas: 15000 (esperado: 139,620)


In [36]:
import pandas as pd
import random
from faker import Faker
from datetime import datetime
from sqlalchemy import create_engine

# Inicializar Faker
fake = Faker('es_ES')

# ------------------ ARCHIVOS CSV ------------------
ruta_proveedores = r"C:\Users\Usuario\OneDrive\Vs code\superMercado\proveedores.csv"
ruta_empleados = r"C:\Users\Usuario\OneDrive\Vs code\superMercado\empleados.csv"

proveedores_df = pd.read_csv(ruta_proveedores)
empleados_df = pd.read_csv(ruta_empleados)

# ------------------ GENERAR ORDENES ------------------
ordenes = []
cantidad_ordenes = 30000
estados = ['pendiente', 'recibida', 'cancelada']

for _ in range(cantidad_ordenes):
    fecha = fake.date_between(start_date='-5y', end_date='today')  # Últimos 5 años
    total = round(random.uniform(1000, 50000), 2)
    estado = random.choices(estados, weights=[0.3, 0.6, 0.1])[0]  # Más probabilidad de 'recibida'
    id_proveedor = int(proveedores_df.sample(1).iloc[0]['id_proveedor'])
    id_empleado = int(empleados_df.sample(1).iloc[0]['id_empleado'])

    ordenes.append({
        'fecha': fecha,
        'total': total,
        'estado': estado,
        'id_proveedor': id_proveedor,
        'id_empleado': id_empleado
    })

ordenes_df = pd.DataFrame(ordenes)

# ------------------ CONEXIÓN MYSQL ------------------
usuario = "root"
contraseña = "Datasciencie98"  # Reemplaza con tu contraseña real
host = "localhost"
base_datos = "supermercado"

conexion = create_engine(f"mysql+pymysql://{usuario}:{contraseña}@{host}/{base_datos}")

# ------------------ INSERTAR EN MYSQL ------------------
ordenes_df.to_sql('ordenes_compra', con=conexion, if_exists='append', index=False)

print(f"✅ Órdenes generadas: {len(ordenes_df)}")
print("🚀 Datos insertados correctamente en la tabla 'ordenes_compra'")


✅ Órdenes generadas: 30000
🚀 Datos insertados correctamente en la tabla 'ordenes_compra'


In [3]:
import pandas as pd
import random
from faker import Faker
from datetime import datetime, timedelta
import os

# --- Configuración Inicial ---
faker = Faker('es_CO')
output_file = "ventas_generadas.csv"

# Parámetros
total_registros = 3_000_000
bloque = 100_000  # Puedes ajustar este número si tu máquina es más potente
num_bloques = total_registros // bloque

# --- Cargar datos base ---
ruta_base = "C:/Users/Usuario/OneDrive/Vs code/superMercado/"

clientes = pd.read_csv(ruta_base + "clientes.csv")
empleados = pd.read_csv(ruta_base + "empleados.csv")
sucursales = pd.read_csv(ruta_base + "sucursales.csv")

id_clientes = clientes['id_cliente'].tolist()
id_empleados = empleados['id_empleado'].tolist()
id_sucursales = sucursales['id_sucursal'].tolist()

# Borrar archivo anterior si existe
if os.path.exists(output_file):
    os.remove(output_file)

# --- Función para generar bloque de ventas ---
def generar_bloque_ventas(bloque_size, start_id=1):
    ventas = []

    for i in range(bloque_size):
        id_venta = start_id + i

        # Fecha aleatoria en los últimos 2 años
        fecha = faker.date_time_between(start_date='-2y', end_date='now')

        # Total aleatorio entre $5.000 y $1.000.000
        total = round(random.uniform(5000, 1_000_000), 2)

        # Método de pago con distribución
        metodo_pago = random.choices(
            ['tarjeta', 'efectivo', 'transferencia'],
            weights=[0.6, 0.3, 0.1],
            k=1
        )[0]

        # Cliente (90% con cliente, 10% anónima)
        id_cliente = random.choice(id_clientes) if random.random() < 0.9 else None

        # Empleado y sucursal
        id_empleado = random.choice(id_empleados)
        id_sucursal = random.choice(id_sucursales)

        ventas.append([
            id_venta, fecha, total, metodo_pago, id_cliente, id_empleado, id_sucursal
        ])

    return pd.DataFrame(ventas, columns=[
        "id_venta", "fecha", "total", "metodo_pago", "id_cliente", "id_empleado", "id_sucursal"
    ])

# --- Generar datos por bloques ---
print("Generando ventas...")

start_id = 1
for i in range(num_bloques):
    df_bloque = generar_bloque_ventas(bloque, start_id)
    df_bloque.to_csv(output_file, mode='a', index=False, header=not os.path.exists(output_file))
    start_id += bloque
    print(f"Bloque {i+1}/{num_bloques} generado ({start_id-1} ventas)")

# Si sobran algunos registros
resto = total_registros % bloque
if resto > 0:
    df_resto = generar_bloque_ventas(resto, start_id)
    df_resto.to_csv(output_file, mode='a', index=False, header=False)
    print(f"Bloque final de {resto} ventas generado")

print("✅ Generación completada. Archivo guardado en:", output_file)


Generando ventas...
Bloque 1/30 generado (100000 ventas)
Bloque 2/30 generado (200000 ventas)
Bloque 3/30 generado (300000 ventas)
Bloque 4/30 generado (400000 ventas)
Bloque 5/30 generado (500000 ventas)
Bloque 6/30 generado (600000 ventas)
Bloque 7/30 generado (700000 ventas)
Bloque 8/30 generado (800000 ventas)
Bloque 9/30 generado (900000 ventas)
Bloque 10/30 generado (1000000 ventas)
Bloque 11/30 generado (1100000 ventas)
Bloque 12/30 generado (1200000 ventas)
Bloque 13/30 generado (1300000 ventas)
Bloque 14/30 generado (1400000 ventas)
Bloque 15/30 generado (1500000 ventas)
Bloque 16/30 generado (1600000 ventas)
Bloque 17/30 generado (1700000 ventas)
Bloque 18/30 generado (1800000 ventas)
Bloque 19/30 generado (1900000 ventas)
Bloque 20/30 generado (2000000 ventas)
Bloque 21/30 generado (2100000 ventas)
Bloque 22/30 generado (2200000 ventas)
Bloque 23/30 generado (2300000 ventas)
Bloque 24/30 generado (2400000 ventas)
Bloque 25/30 generado (2500000 ventas)
Bloque 26/30 generado (

In [1]:
pip install pymysql sqlalchemy pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
from sqlalchemy import create_engine

# ------------------ CONEXIÓN MYSQL ------------------
usuario = "root"
contraseña = "Datasciencie98"  # Reemplaza con tu contraseña real
host = "localhost"
base_datos = "supermercado"

conexion = create_engine(f"mysql+pymysql://{usuario}:{contraseña}@{host}/{base_datos}")

# ------------------ CARGAR CSV ------------------
archivo_csv = "ventas_generadas.csv"
ventas_df = pd.read_csv(archivo_csv, parse_dates=['fecha'])

# ------------------ INSERTAR EN MYSQL ------------------
ventas_df.to_sql('ventas', con=conexion, if_exists='append', index=False)

print(f"✅ Ventas generadas: {len(ventas_df)}")
print("🚀 Datos insertados correctamente en la tabla 'ventas'")

✅ Ventas generadas: 3000000
🚀 Datos insertados correctamente en la tabla 'ventas'


In [4]:
import pandas as pd
import random
from faker import Faker
import os

faker = Faker('es_CO')

# ------------------ CONFIGURACIÓN DE RUTA ------------------
ruta_base = "C:/Users/Usuario/OneDrive/Vs code/superMercado/"
archivo_salida = ruta_base + "detalle_orden_compra.csv"

# ------------------ CARGAR DATOS BASE ------------------
productos_df = pd.read_csv(ruta_base + "productos.csv")
ordenes_df = pd.read_csv(ruta_base + "ordenes_compra.csv")

# Normalizar encabezados
productos_df.columns = productos_df.columns.str.strip().str.lower()
ordenes_df.columns = ordenes_df.columns.str.strip().str.lower()

# Obtener listas de IDs
id_productos = productos_df['id_producto'].tolist()
id_ordenes = ordenes_df['id_orden'].tolist()

# Crear diccionario de precios
if 'precio' in productos_df.columns:
    precios_dict = productos_df.set_index('id_producto')['precio'].to_dict()
else:
    precios_dict = {pid: round(random.uniform(2000, 80000), 2) for pid in id_productos}

# ------------------ PARÁMETROS DE BLOQUE ------------------
bloque = 100_000   # Órdenes por bloque
total_ordenes = len(id_ordenes)
num_bloques = total_ordenes // bloque

# Eliminar archivo anterior si existe
if os.path.exists(archivo_salida):
    os.remove(archivo_salida)

# ------------------ FUNCIÓN GENERADORA ------------------
def generar_detalles_orden(bloque_ordenes, start_id_detalle):
    detalles = []
    detalle_id = start_id_detalle

    for id_orden in bloque_ordenes:
        num_items = random.randint(2, 6)
        productos_elegidos = random.sample(id_productos, num_items)

        for id_producto in productos_elegidos:
            cantidad = random.randint(1, 10)
            precio_unitario = precios_dict.get(id_producto, round(random.uniform(2000, 80000), 2))

            detalles.append([
                detalle_id,
                cantidad,
                precio_unitario,
                id_orden,
                id_producto
            ])
            detalle_id += 1

    df_detalle = pd.DataFrame(detalles, columns=[
        'id_detalle_orden', 'cantidad', 'precio_unitario', 'id_orden', 'id_producto'
    ])
    return df_detalle, detalle_id

# ------------------ GENERACIÓN POR BLOQUES ------------------
print("🛠 Generando detalles de órdenes por bloques...")
id_detalle_actual = 1

for i in range(num_bloques):
    inicio = i * bloque
    fin = inicio + bloque
    bloque_ordenes = id_ordenes[inicio:fin]

    df_bloque, id_detalle_actual = generar_detalles_orden(bloque_ordenes, id_detalle_actual)
    df_bloque.to_csv(archivo_salida, mode='a', header=not os.path.exists(archivo_salida), index=False)

    print(f"✅ Bloque {i+1}/{num_bloques} guardado. Último ID: {id_detalle_actual - 1}")

# Procesar resto si existe
resto = total_ordenes % bloque
if resto > 0:
    bloque_ordenes = id_ordenes[-resto:]
    df_bloque, id_detalle_actual = generar_detalles_orden(bloque_ordenes, id_detalle_actual)
    df_bloque.to_csv(archivo_salida, mode='a', header=not os.path.exists(archivo_salida), index=False)
    print(f"✅ Bloque final guardado. Último ID: {id_detalle_actual - 1}")

print(f"🎉 Archivo final creado: {archivo_salida} con aproximadamente {id_detalle_actual - 1} registros.")


🛠 Generando detalles de órdenes por bloques...
✅ Bloque final guardado. Último ID: 4040
🎉 Archivo final creado: C:/Users/Usuario/OneDrive/Vs code/superMercado/detalle_orden_compra.csv con aproximadamente 4040 registros.


In [5]:
import pandas as pd
from sqlalchemy import create_engine

# ------------------ CONEXIÓN MYSQL ------------------
usuario = "root"
contraseña = "Datasciencie98"  # Cambia si usas otra
host = "localhost"
base_datos = "supermercado"

conexion = create_engine(f"mysql+pymysql://{usuario}:{contraseña}@{host}/{base_datos}")

# ------------------ CARGAR ARCHIVO CSV ------------------
archivo_csv = "C:/Users/Usuario/OneDrive/Vs code/superMercado/detalle_orden_compra.csv"
detalle_df = pd.read_csv(archivo_csv)

# ------------------ IMPORTAR A MYSQL ------------------
detalle_df.to_sql('detalle_orden_compra', con=conexion, if_exists='append', index=False)

print(f"✅ Registros insertados: {len(detalle_df)}")
print("🚀 Datos insertados correctamente en la tabla 'detalle_orden_compra'")


✅ Registros insertados: 4040
🚀 Datos insertados correctamente en la tabla 'detalle_orden_compra'
